In [1]:
import pandas as pd

In [2]:
# Load CSV files
students_df = pd.read_csv('Resources/students_complete.csv')
schools_df = pd.read_csv('Resources/schools_complete.csv')

In [3]:
# Merge DataFrames
school_data_complete = pd.merge(students_df, schools_df, how='left', on=['school_name'])

In [4]:
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [5]:
# District Summary
total_schools = schools_df['school_name'].nunique()
total_students = students_df['Student ID'].nunique()
total_budget = schools_df['budget'].sum()
average_math_score = students_df['math_score'].mean()
average_reading_score = students_df['reading_score'].mean()

passing_math = students_df[students_df['math_score'] >= 70]['Student ID'].count()
percent_passing_math = (passing_math / total_students) * 100

passing_reading = students_df[students_df['reading_score'] >= 70]['Student ID'].count()
percent_passing_reading = (passing_reading / total_students) * 100

overall_passing = students_df[(students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70)]['Student ID'].count()
percent_overall_passing = (overall_passing / total_students) * 100
district_summary = pd.DataFrame({
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [average_math_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Math': [percent_passing_math],
    '% Passing Reading': [percent_passing_reading],
    '% Overall Passing': [percent_overall_passing]
})

In [6]:
display(district_summary)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [7]:
# School Summary
school_summary = school_data_complete.groupby('school_name').agg({
    'type': 'first',
    'Student ID': 'count',
    'budget': 'first',
    'math_score': 'mean',
    'reading_score': 'mean',
    'math_pass': lambda x: (x >= 70).sum(),
    'reading_pass': lambda x: (x >= 70).sum(),
    'overall_pass': lambda x: ((x['math_score'] >= 70) & (x['reading_score'] >= 70)).sum()
}).rename(columns={
    'type': 'School Type',
    'Student ID': 'Total Students',
    'budget': 'Total School Budget',
    'math_score': 'Average Math Score',
    'reading_score': 'Average Reading Score',
    'math_pass': '% Passing Math',
    'reading_pass': '% Passing Reading',
    'overall_pass': '% Overall Passing'
})

school_summary['Per Student Budget'] = school_summary['Total School Budget'] / school_summary['Total Students']


KeyError: "Column(s) ['math_pass', 'overall_pass', 'reading_pass'] do not exist"

In [ ]:
display(school_summary)

In [ ]:
# Top Performing Schools
top_schools = school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

In [ ]:
# Bottom Performing Schools
bottom_schools = school_summary.sort_values(by='% Overall Passing').head(5)

In [ ]:
# Math Scores by Grade
math_scores_by_grade = pd.pivot_table(students_df, values='math_score', index='school_name', columns='grade', aggfunc='mean')

In [ ]:
# Reading Scores by Grade
reading_scores_by_grade = pd.pivot_table(students_df, values='reading_score', index='school_name', columns='grade', aggfunc='mean')

In [ ]:
# Scores by School Spending
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending_df = school_summary.copy()
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=labels)

spending_summary = school_spending_df.groupby('Spending Ranges (Per Student)').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean',
    '% Overall Passing': 'mean'
})

In [ ]:
# Scores by School Size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary = school_summary.copy()
per_school_summary['School Size'] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels)

size_summary = per_school_summary.groupby('School Size').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean',
    '% Overall Passing': 'mean'
})

In [ ]:
# Scores by School Type
type_summary = per_school_summary.groupby('School Type').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean',
    '% Overall Passing': 'mean'
})